Experiment: Concatenates together last 4 attention distribution
------
concatenates together the last four attention distributions; using only one attention head.

In [1]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [2]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
attention_dataloader = dataset.return_dataloader()
attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/03/2019 13:17:54 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
500it [00:00, 6771.78it/s]
06/03/2019 13:17:55 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/03/2019 13:17:55 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../../../tasks/bias_classification/results/cache/9c41111e2de8

In [ ]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

In [3]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [5]:
# Extracting indices of bias labels 
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)
# windowing indices
windowed_data, removed_indices = window_attention_dist(biased_tok_dist, bias_indices, window_size=params.final_task['window_size'])

In [9]:
concat_dist = concat_attention_dist(windowed_data)
print(concat_dist)

[tensor([5.9950e-08, 1.2114e-06, 2.9103e-03, 3.0207e-03, 5.0348e-03, 3.5970e-02,
        7.3431e-04, 5.1745e-04, 1.9713e-03, 4.4441e-04, 1.2841e-04, 1.9021e-06,
        1.2013e-05, 4.0411e-04, 3.4309e-03, 1.7463e-03, 7.5755e-01, 1.8091e-04,
        2.6013e-04, 6.5182e-04, 1.4439e-04, 9.2663e-06, 7.8744e-08, 5.3600e-07,
        1.7734e-05, 3.1926e-04, 1.5003e-04, 9.5423e-01, 1.0130e-05, 2.6103e-06,
        1.8001e-05, 5.4418e-06, 1.0821e-06, 2.1346e-08, 1.1191e-07, 5.5400e-06,
        4.7462e-04, 4.6073e-05, 3.0411e-01, 1.1829e-06, 6.1681e-08, 2.4927e-06,
        1.4075e-06, 2.7551e-07], device='cuda:0'), tensor([9.2869e-07, 1.3182e-04, 3.7884e-04, 2.9610e-04, 2.4734e-04, 1.9021e-02,
        1.6256e-03, 1.9532e-04, 8.2146e-05, 5.1261e-05, 3.5585e-05, 7.7072e-06,
        3.6703e-04, 5.7667e-05, 3.0682e-03, 2.1265e-04, 2.0546e-02, 9.1299e-05,
        3.5094e-05, 9.4349e-06, 1.2503e-05, 2.2317e-05, 2.0931e-06, 7.9225e-04,
        1.9702e-05, 1.3052e-02, 3.9053e-06, 9.6244e-01, 4.9393e-06, 

In [10]:
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(concat_dist), "attention_dist")
dataset.shuffle_data()

Run the cell below to reset params 
-----

In [11]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [12]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 44, 'hidden_dim': 10, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [15]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [16]:
statistics

{'auc': [(0.6752843679348468, 0.8509239577269152), 0.7677060136452627],
 'accuracy': [(0.6395348837209303, 0.8023255813953488), 0.7262790697674417]}

New run 

In [35]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [36]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 11, 'hidden_dim': 3, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [22]:
statistics

{'auc': [(0.49685111400227683, 0.8255225075881711), 0.7107909485455589],
 'accuracy': [(0.5869186046511627, 0.7502906976744186), 0.6730232558139536]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [23]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [24]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [25]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [26]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.5602870774269103


In [27]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [28]:
print(max_auc)

0.6637993874070985


In [31]:
print(min_auc)

0.6379312454511352
